In [1492]:
import pandas as pd
import matplotlib as plt
import numpy as np
pd.set_option('display.max_columns', 500)


In [1493]:
hawkeye = pd.read_stata('/Users/lucaslippman/Daniel/MergeDaniel/HKdata_3matches.dta')

challenge = pd.read_stata('/Users/lucaslippman/Daniel/MergeDaniel/Challenges_3matches.dta')


In [ ]:
big= pd.read_stata('/Users/lucaslippman/Daniel/MergeDaniel/Extract data for David 5 July 2023.dta')

In [1494]:
hawkeye['number_hawkeye'] = hawkeye.index
challenge['number_challenge'] = challenge.index
hawkeye['hits'] = hawkeye.apply(lambda row: row['server'] if row['puttheball'] == 'server' else row['receiver'], axis=1)
hawkeye["set"] = hawkeye["scoresetserver"] + hawkeye["scoresetreceiver"] +1
hawkeye["sum_games"] = hawkeye["scoregameserver"] + hawkeye["scoregamereceiver"]
hawkeye["dist_abs"] = abs(hawkeye["dist"])
challenge["call"] = challenge["hawk_in"].apply(lambda x: "in" if x == 1 else "out")
challenge["distance"] = challenge["distance_mm"] / 1000


In [1495]:
def calculate_sum(row):
    set_number = int(row['set'])

    if set_number == 1:
        sum_score = row['set1_chal'] + row['set1_opp']
        if ((sum_score == 10) and ((row['set1_chal'] == 4 and row['set1_opp'] == 6) or (row['set1_chal'] == 6 and row['set1_opp'] == 4))):
            return 0
    elif set_number == 2:
        sum_score = row['set2_chal'] + row['set2_opp']
        if ((sum_score == 10) and ((row['set2_chal'] == 4 and row['set2_opp'] == 6) or (row['set2_chal'] == 6 and row['set2_opp'] == 4))):
            return 0
    elif set_number == 3:
        sum_score = row['set3_chal'] + row['set3_opp']
        if ((sum_score == 10) and ((row['set3_chal'] == 4 and row['set3_opp'] == 6) or (row['set3_chal'] == 6 and row['set3_opp'] == 4))):
            return 0
    else:
        return 0

    return sum_score

challenge['sum_games'] = challenge.apply(calculate_sum, axis=1)
challenge["out"] = challenge["call"].replace({'out': 1, 'in': 0})
challenge['sum_games'] = challenge['sum_games'].fillna(0)



In [1496]:

challenge['challenger_name'] = challenge['challenger_name'].str.replace("López", "LOPEZ")
challenge['opponent_name'] = challenge['opponent_name'].str.replace("López", "LOPEZ")
challenge['hit'] = challenge.apply(lambda row: (row['challenger_name'].split()[-1].replace("LÓPEZ", "LOPEZ")).upper() if row['call_in'] == 0 else (row['opponent_name'].split()[-1].replace("LÓPEZ", "LOPEZ")).upper(), axis=1)



In [1497]:
def tennis_points(value):
    value = int(value)
    if value == 0:
        return 0
    elif value == 1:
        return 15
    elif value == 2:
        return 30
    elif value == 3:
        return 40
    else:
        return 50

# Split the 'score' column into two columns
hawkeye[['server_score', 'receiver_score']] = hawkeye['score'].str.split(' ', expand=True)

# Convert the tennis points to their actual values
hawkeye[['server_score', 'receiver_score']] = hawkeye[['server_score', 'receiver_score']].applymap(tennis_points)


In [1498]:
hawkeye['comb_game_x'] = hawkeye['server_score'] + hawkeye['receiver_score']
hawkeye['comb_game_x'] = hawkeye['comb_game_x'].apply(lambda x: min(x, 90))

# Add the 'comb_game_y' column based on the conditions
challenge['comb_game_y'] = challenge.apply(lambda row: 90 if row['game_chal'] == 50 or row['game_opp'] == 50 else row['game_chal'] + row['game_opp'], axis=1)

hawkeye['comb_game_x'] = hawkeye['comb_game_x'].astype(float)
challenge['comb_game_y'] = challenge['comb_game_y'].astype(float)
challenge['comb_game_y'] = challenge['comb_game_y'].fillna(0)



In [1499]:
def merge_condition(row):
    return (
        row["set_x"] == row["set_y"]
        and row["sum_games_x"] == row["sum_games_y"]
        and row['tournament_x'] == row['tournament_y']
        and row['hits'] == row['hit']

    )

    

# Perform a cross join to combine all possible pairs of rows
hawkeye["key"] = 1
challenge["key"] = 1
merged_df = pd.merge(hawkeye, challenge, on="key").drop("key", axis=1)

# Filter the rows based on the merge_condition
merged_df["merge_condition"] = merged_df.apply(merge_condition, axis=1)
merged_df = merged_df[merged_df["merge_condition"]]

# Drop the duplicate columns and the merge_condition column
merged_df = merged_df.drop(
    columns=["set_y", "sum_games_y", "out_y", "merge_condition"]
)

# Rename the 'number' column from the 'challenge' DataFrame to avoid confusion
cols_to_move = ['number_hawkeye', 'number_challenge','dist_abs','score','game_chal','game_opp', 'hawk_in','challenge_won','comb_game_x','comb_game_y']
merged_df = merged_df[[col for col in merged_df.columns if col not in cols_to_move] + cols_to_move]
merged_df = merged_df.sort_values(by="number_challenge")



In [1500]:
def select_desired_row(group):
    # Select rows where 'comb_game_x' and 'comb_game_y' are equal
    same_comb_game = group[group['comb_game_x'] == group['comb_game_y']]
    
    # If there are suitable 'comb_game_x' and 'comb_game_y' values
    if not same_comb_game.empty:
        # Find the index of the row with the minimum difference between 'dist_abs' and 'distance' for suitable rows
        min_diff_index = (same_comb_game['dist_abs'] - same_comb_game['distance']).abs().idxmin()
        # Return the row with the minimum difference for suitable rows
        return same_comb_game.loc[min_diff_index]
    else:
        # If either 'comb_game_x' or 'comb_game_y' has missing values (NaN) in the group, or there are no suitable rows
        if group['comb_game_x'].isna().any() or group['comb_game_y'].isna().any() or same_comb_game.empty:
            # Find the index of the row with the minimum difference between 'dist_abs' and 'distance'
            min_diff_index = (group['dist_abs'] - group['distance']).abs().idxmin()
            # Return the row with the minimum difference
            return group.loc[min_diff_index]

# Group the dataframe by 'number_challenges_2' and apply the 'select_desired_row' function to each group


strict = merged_df.groupby('number_challenge').apply(select_desired_row).reset_index(drop=True)

In [1501]:
strict

,server,receiver,scorer,impact,x1,y1,foldername_x,tournament_x,iiid,gender,scoreserver,scorereceiver,iid1,idmatch,idinpoint,bounce,iid2,scorer2,volley,puttheball,doublebounce,alreadydoublebounce,issueplayerside,dist,out_x,alreadyout,errorscoregame,inside,player1,scoregameplayer1,scoregameplayer2,scoresetserver,scoresetreceiver,scoregameserver,scoregamereceiver,hits,set_x,sum_games_x,server_score,receiver_score,tournament_id,challenger1_id,round,opponent1_id,match_time,set1_chal,set1_opp,set2_chal,set2_opp,set3_chal,set3_opp,match_challenge_count,challenges_left,call_in,distance_mm,line,player_stopped,ump_comments,umpire_challenger_ord,tournament_y,challenger_name,opponent_name,min,max,foldername_y,serve,chal_served_point,call,distance,hit,number_hawkeye,number_challenge,dist_abs,score,game_chal,game_opp,hawk_in,challenge_won,comb_game_x,comb_game_y
0,LOPEZ,DAVYDENKO,-1,0.0,-6.651841,-1.147098,SemiLopezDavydenko,DubaiMens08,570970.0,male,1,3,127699.0,573,1,1,92500.0,-1,0,server,0,0,NaN,-0.123161,1,0,19.047619,0.0,0.0,3.0,1.0,1.0,1.0,1.0,2.0,LOPEZ,3.0,3.0,15,40,6,68.0,4,202.0,107,6.0,4.0,4.0,6.0,1.0,2.0,7,1,0,121.0,RS,0,,2,DubaiMens08,Feliciano LOPEZ,Nikolay Davydenko,68.0,202.0,SemiLopezDavydenko,1.0,1.0,out,0.121,LOPEZ,781,0,0.123161,1 3,15.0,40.0,0,0,55.0,55.0
1,DAVYDENKO,LOPEZ,1,0.0,-6.414097,2.683151,SemiLopezDavydenko,DubaiMens08,570793.0,male,1,0,127654.0,573,1,1,92466.0,1,0,server,0,0,NaN,0.124471,0,0,0.000000,1.0,1.0,5.0,4.0,0.0,1.0,4.0,4.0,DAVYDENKO,2.0,8.0,15,0,6,68.0,4,202.0,82,6.0,4.0,4.0,4.0,NaN,NaN,4,2,1,132.0,RS,0,,2,DubaiMens08,Feliciano LOPEZ,Nikolay Davydenko,68.0,202.0,SemiLopezDavydenko,1.0,0.0,in,0.132,DAVYDENKO,604,1,0.124471,1 0,0.0,15.0,1,0,15.0,15.0
2,LOPEZ,DAVYDENKO,0,0.0,6.629273,3.591878,SemiLopezDavydenko,DubaiMens08,570225.0,male,0,2,127501.0,573,1,1,92358.0,0,0,server,0,0,NaN,-0.092159,1,0,18.181818,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,LOPEZ,1.0,1.0,0,30,6,68.0,4,202.0,4,0.0,1.0,NaN,NaN,NaN,NaN,1,2,0,86.0,LS,0,,2,DubaiMens08,Feliciano LOPEZ,Nikolay Davydenko,68.0,202.0,SemiLopezDavydenko,1.0,1.0,out,0.086,LOPEZ,37,2,0.092159,0 2,0.0,30.0,0,0,30.0,30.0
3,LOPEZ,DAVYDENKO,-1,10.0,-7.182900,-2.882504,SemiLopezDavydenko,DubaiMens08,570923.0,male,2,0,127683.0,573,5,4,92487.0,-1,0,receiver,0,0,NaN,1.271605,0,0,0.000000,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,DAVYDENKO,3.0,1.0,30,0,6,68.0,4,202.0,98,6.0,4.0,4.0,6.0,0.0,1.0,5,2,1,4.0,RB,0,,2,DubaiMens08,Feliciano LOPEZ,Nikolay Davydenko,68.0,202.0,SemiLopezDavydenko,0.0,NaN,out,0.004,DAVYDENKO,734,3,1.271605,2 0,15.0,15.0,0,1,30.0,30.0
4,DAVYDENKO,LOPEZ,-1,18.0,10.686390,-4.157564,SemiLopezDavydenko,DubaiMens08,570420.0,male,0,1,127553.0,573,7,7,92395.0,-1,0,server,0,0,NaN,-0.009434,1,0,0.000000,0.0,1.0,4.0,5.0,0.0,0.0,4.0,4.0,DAVYDENKO,1.0,8.0,0,15,6,202.0,4,68.0,30,4.0,4.0,NaN,NaN,NaN,NaN,2,1,0,6.0,LF,0,,1,DubaiMens08,Nikolay Davydenko,Feliciano LOPEZ,68.0,202.0,SemiLopezDavydenko,0.0,NaN,out,0.006,DAVYDENKO,231,4,0.009434,0 1,0.0,15.0,0,0,15.0,15.0
5,LOPEZ,DAVYDENKO,-1,12.0,-9.783928,4.180157,SemiLopezDavydenko,DubaiMens08,570958.0,male,0,0,127694.0,573,5,5,92495.0,-1,0,server,0,0,NaN,-0.029688,1,0,19.047619,0.0,0.0,3.0,1.0,1.0,1.0,1.0,2.0,LOPEZ,3.0,3.0,0,0,6,68.0,4,202.0,105,6.0,4.0,4.0,6.0,1.0,2.0,6,2,0,26.0,RN,1,,2,DubaiMens08,Feliciano LOPEZ,Nikolay Davydenko,68.0,202.0,SemiLopezDavydenko,0.0,NaN,out,0.026,LOPEZ,769,5,0.029688,0 0,NaN,NaN,0,0,0.0,0.0
6,DAVYDENKO,LOPEZ,0,0.0,-6.606727,3.722899,SemiLopezDavydenko,DubaiMens08,570531.0,male,2,3,127582.0,573,1,1,92415.0,1,0,server,0,0,NaN,-0.074273,1,0,0.000000,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,DAVYDENKO,2.0,0.0,30,40,6,68.0,4,202.0,46,NaN,NaN,6.0,4.0,NaN,NaN,3,2,1,66.0,RS,0,,2,DubaiMens08,Feliciano LOPEZ,Nikolay Davydenko,68.0,202.0,SemiLopezDavydenko,1.0,0.0,out,0.066,DAVYDENKO,342,6,0.074273,2 3,40.0,30.0,0,1,70.0,70.0
7,DAVYDENKO,LOPEZ,-1,6.0,-12.049870,-2.992609,SemiLopezDavydenko,DubaiMens08,570367.0,male,0,1,127537.0,573,3,3,92384.0,-1,0,server,0,0,NaN,-0.050660,1,0,0.000000,0.0,1.0,4.0,3.0,0.0,0.0,4.0,2.0,DAVYD

In [1502]:
david = pd.read_csv('/Users/lucaslippman/Daniel/Merge_Lucas.csv')


STRICT

In [1503]:
hawkeye_ids = david['HAWKEYE_ID'].unique()
number_hawkeye = strict['number_hawkeye'].unique()

# Calculate the total number of unique values
total_unique_values = len(hawkeye_ids)

# Get the intersection of the two sets of unique values
common_values = len(set(hawkeye_ids) & set(number_hawkeye))

# Compute the accuracy
accuracy = common_values / total_unique_values

print(f"Accuracy: {accuracy * 100}%")
print(common_values)


Accuracy: 67.64705882352942%
23


RELAX1


In [1504]:
def select_desired_row(group):
    # Select rows where 'comb_game_x' and 'comb_game_y' are within a range of +-20
    same_comb_game = group[abs(group['comb_game_x'] - group['comb_game_y']) <= 20]
    
    # If there are suitable 'comb_game_x' and 'comb_game_y' values
    if not same_comb_game.empty:
        # Find the index of the row with the minimum difference between 'dist_abs' and 'distance' for suitable rows
        min_diff_index = (same_comb_game['dist_abs'] - same_comb_game['distance']).abs().idxmin()
        # Return the row with the minimum difference for suitable rows
        return same_comb_game.loc[min_diff_index]
    else:
        # If either 'comb_game_x' or 'comb_game_y' has missing values (NaN) in the group, or there are no suitable rows
        if group['comb_game_x'].isna().any() or group['comb_game_y'].isna().any() or same_comb_game.empty:
            # Find the index of the row with the minimum difference between 'dist_abs' and 'distance'
            min_diff_index = (group['dist_abs'] - group['distance']).abs().idxmin()
            # Return the row with the minimum difference
            return group.loc[min_diff_index]

# Group the dataframe by 'number_challenges_2' and apply the 'select_desired_row' function to each group
relax1 = merged_df.groupby('number_challenge').apply(select_desired_row).reset_index(drop=True)
hawkeye_ids = david['HAWKEYE_ID'].unique()
number_hawkeye = relax1['number_hawkeye'].unique()

# Calculate the total number of unique values
total_unique_values = len(hawkeye_ids)

# Get the intersection of the two sets of unique values
common_values = len(set(hawkeye_ids) & set(number_hawkeye))

# Compute the accuracy
accuracy = common_values / total_unique_values

print(f"Accuracy: {accuracy * 100}%")
print(common_values)


Accuracy: 82.35294117647058%
28


RELAX2

In [1505]:
def merge_condition(row):
    return (
        row["set_x"] == row["set_y"]
        and abs(row["sum_games_x"] - row["sum_games_y"]) <= 1
        and row['tournament_x'] == row['tournament_y']
        and row['hits'] == row['hit']
    )
    

# Perform a cross join to combine all possible pairs of rows
hawkeye["key"] = 1
challenge["key"] = 1
merged_df = pd.merge(hawkeye, challenge, on="key").drop("key", axis=1)

# Filter the rows based on the merge_condition
merged_df["merge_condition"] = merged_df.apply(merge_condition, axis=1)
merged_df = merged_df[merged_df["merge_condition"]]


# Rename the 'number' column from the 'challenge' DataFrame to avoid confusion
cols_to_move = ['number_hawkeye', 'number_challenge','dist_abs','score','game_chal','game_opp', 'hawk_in','challenge_won','comb_game_x','comb_game_y']
merged_df = merged_df[[col for col in merged_df.columns if col not in cols_to_move] + cols_to_move]
merged_df = merged_df.sort_values(by="number_challenge")
def select_desired_row(group):
    # Select rows where 'comb_game_x' and 'comb_game_y' are within a range of +-20
    same_comb_game = group[abs(group['comb_game_x'] - group['comb_game_y']) <= 20]
    
    # If there are suitable 'comb_game_x' and 'comb_game_y' values
    if not same_comb_game.empty:
        # Find the index of the row with the minimum difference between 'dist_abs' and 'distance' for suitable rows
        min_diff_index = (same_comb_game['dist_abs'] - same_comb_game['distance']).abs().idxmin()
        # Return the row with the minimum difference for suitable rows
        return same_comb_game.loc[min_diff_index]
    else:
        # If either 'comb_game_x' or 'comb_game_y' has missing values (NaN) in the group, or there are no suitable rows
        if group['comb_game_x'].isna().any() or group['comb_game_y'].isna().any() or same_comb_game.empty:
            # Find the index of the row with the minimum difference between 'dist_abs' and 'distance'
            min_diff_index = (group['dist_abs'] - group['distance']).abs().idxmin()
            # Return the row with the minimum difference
            return group.loc[min_diff_index]

# Group the dataframe by 'number_challenges_2' and apply the 'select_desired_row' function to each group
relax2 = merged_df.groupby('number_challenge').apply(select_desired_row).reset_index(drop=True)
hawkeye_ids = david['HAWKEYE_ID'].unique()
number_hawkeye = relax2['number_hawkeye'].unique()

# Calculate the total number of unique values
total_unique_values = len(hawkeye_ids)

# Get the intersection of the two sets of unique values
common_values = len(set(hawkeye_ids) & set(number_hawkeye))

# Compute the accuracy
accuracy = common_values / total_unique_values

print(f"Accuracy: {accuracy * 100}%")
print(common_values)


Accuracy: 85.29411764705883%
29


In [1506]:
relax2.to_csv('/Users/lucaslippman/Daniel/relax2.csv')

Relax3

In [1507]:
def merge_condition(row):
    return (
        row["set_x"] == row["set_y"]
        and abs(row["sum_games_x"] - row["sum_games_y"]) <= 2
        and row['tournament_x'] == row['tournament_y']
        and row['hits'] == row['hit']
    )
    

# Perform a cross join to combine all possible pairs of rows
hawkeye["key"] = 1
challenge["key"] = 1
merged_df = pd.merge(hawkeye, challenge, on="key").drop("key", axis=1)

# Filter the rows based on the merge_condition
merged_df["merge_condition"] = merged_df.apply(merge_condition, axis=1)
merged_df = merged_df[merged_df["merge_condition"]]


# Rename the 'number' column from the 'challenge' DataFrame to avoid confusion
cols_to_move = ['number_hawkeye', 'number_challenge','dist_abs','score','game_chal','game_opp', 'hawk_in','challenge_won','comb_game_x','comb_game_y']
merged_df = merged_df[[col for col in merged_df.columns if col not in cols_to_move] + cols_to_move]
merged_df = merged_df.sort_values(by="number_challenge")
def select_desired_row(group):
    # Select rows where 'comb_game_x' and 'comb_game_y' are within a range of +-20
    same_comb_game = group[abs(group['comb_game_x'] - group['comb_game_y']) <= 1000]
    
    # If there are suitable 'comb_game_x' and 'comb_game_y' values
    if not same_comb_game.empty:
        # Find the index of the row with the minimum difference between 'dist_abs' and 'distance' for suitable rows
        min_diff_index = (same_comb_game['dist_abs'] - same_comb_game['distance']).abs().idxmin()
        # Return the row with the minimum difference for suitable rows
        return same_comb_game.loc[min_diff_index]
    else:
        # If either 'comb_game_x' or 'comb_game_y' has missing values (NaN) in the group, or there are no suitable rows
        if group['comb_game_x'].isna().any() or group['comb_game_y'].isna().any() or same_comb_game.empty:
            # Find the index of the row with the minimum difference between 'dist_abs' and 'distance'
            min_diff_index = (group['dist_abs'] - group['distance']).abs().idxmin()
            # Return the row with the minimum difference
            return group.loc[min_diff_index]

# Group the dataframe by 'number_challenges_2' and apply the 'select_desired_row' function to each group
relax3 = merged_df.groupby('number_challenge').apply(select_desired_row).reset_index(drop=True)
hawkeye_ids = david['HAWKEYE_ID'].unique()
number_hawkeye = relax3['number_hawkeye'].unique()

# Calculate the total number of unique values
total_unique_values = len(hawkeye_ids)

# Get the intersection of the two sets of unique values
common_values = len(set(hawkeye_ids) & set(number_hawkeye))

# Compute the accuracy
accuracy = common_values / total_unique_values

print(f"Accuracy: {accuracy * 100}%")
print(common_values)


Accuracy: 76.47058823529412%
26


RELAX4

In [1508]:
def merge_condition(row):
    return (
        row["set_x"] == row["set_y"]
        and abs(row["sum_games_x"] - row["sum_games_y"]) <= 1000
        and row['tournament_x'] == row['tournament_y']
        and row['hits'] == row['hit']
    )
    

# Perform a cross join to combine all possible pairs of rows
hawkeye["key"] = 1
challenge["key"] = 1
merged_df = pd.merge(hawkeye, challenge, on="key").drop("key", axis=1)

# Filter the rows based on the merge_condition
merged_df["merge_condition"] = merged_df.apply(merge_condition, axis=1)
merged_df = merged_df[merged_df["merge_condition"]]


# Rename the 'number' column from the 'challenge' DataFrame to avoid confusion
cols_to_move = ['number_hawkeye', 'number_challenge','dist_abs','score','game_chal','game_opp', 'hawk_in','challenge_won','comb_game_x','comb_game_y']
merged_df = merged_df[[col for col in merged_df.columns if col not in cols_to_move] + cols_to_move]
merged_df = merged_df.sort_values(by="number_challenge")
def select_desired_row(group):
    # Select rows where 'comb_game_x' and 'comb_game_y' are within a range of +-20
    same_comb_game = group[abs(group['comb_game_x'] - group['comb_game_y']) <= 1000]
    
    # If there are suitable 'comb_game_x' and 'comb_game_y' values
    if not same_comb_game.empty:
        # Find the index of the row with the minimum difference between 'dist_abs' and 'distance' for suitable rows
        min_diff_index = (same_comb_game['dist_abs'] - same_comb_game['distance']).abs().idxmin()
        # Return the row with the minimum difference for suitable rows
        return same_comb_game.loc[min_diff_index]
    else:
        # If either 'comb_game_x' or 'comb_game_y' has missing values (NaN) in the group, or there are no suitable rows
        if group['comb_game_x'].isna().any() or group['comb_game_y'].isna().any() or same_comb_game.empty:
            # Find the index of the row with the minimum difference between 'dist_abs' and 'distance'
            min_diff_index = (group['dist_abs'] - group['distance']).abs().idxmin()
            # Return the row with the minimum difference
            return group.loc[min_diff_index]

# Group the dataframe by 'number_challenges_2' and apply the 'select_desired_row' function to each group
relax4 = merged_df.groupby('number_challenge').apply(select_desired_row).reset_index(drop=True)
hawkeye_ids = david['HAWKEYE_ID'].unique()
number_hawkeye = relax4['number_hawkeye'].unique()

# Calculate the total number of unique values
total_unique_values = len(hawkeye_ids)

# Get the intersection of the two sets of unique values
common_values = len(set(hawkeye_ids) & set(number_hawkeye))

# Compute the accuracy
accuracy = common_values / total_unique_values

print(f"Accuracy: {accuracy * 100}%")
print(common_values)


Accuracy: 70.58823529411765%
24


RELAX5

In [1509]:
def merge_condition(row):
    return (
        row['tournament_x'] == row['tournament_y']
        and row['hits'] == row['hit']
    )
    

# Perform a cross join to combine all possible pairs of rows
hawkeye["key"] = 1
challenge["key"] = 1
merged_df = pd.merge(hawkeye, challenge, on="key").drop("key", axis=1)

# Filter the rows based on the merge_condition
merged_df["merge_condition"] = merged_df.apply(merge_condition, axis=1)
merged_df = merged_df[merged_df["merge_condition"]]


# Rename the 'number' column from the 'challenge' DataFrame to avoid confusion
cols_to_move = ['number_hawkeye', 'number_challenge','dist_abs','score','game_chal','game_opp', 'hawk_in','challenge_won','comb_game_x','comb_game_y']
merged_df = merged_df[[col for col in merged_df.columns if col not in cols_to_move] + cols_to_move]
merged_df = merged_df.sort_values(by="number_challenge")
def select_desired_row(group):
    # Select rows where 'comb_game_x' and 'comb_game_y' are within a range of +-20
    same_comb_game = group[abs(group['comb_game_x'] - group['comb_game_y']) <= 1000]
    
    # If there are suitable 'comb_game_x' and 'comb_game_y' values
    if not same_comb_game.empty:
        # Find the index of the row with the minimum difference between 'dist_abs' and 'distance' for suitable rows
        min_diff_index = (same_comb_game['dist_abs'] - same_comb_game['distance']).abs().idxmin()
        # Return the row with the minimum difference for suitable rows
        return same_comb_game.loc[min_diff_index]
    else:
        # If either 'comb_game_x' or 'comb_game_y' has missing values (NaN) in the group, or there are no suitable rows
        if group['comb_game_x'].isna().any() or group['comb_game_y'].isna().any() or same_comb_game.empty:
            # Find the index of the row with the minimum difference between 'dist_abs' and 'distance'
            min_diff_index = (group['dist_abs'] - group['distance']).abs().idxmin()
            # Return the row with the minimum difference
            return group.loc[min_diff_index]

# Group the dataframe by 'number_challenges_2' and apply the 'select_desired_row' function to each group
relax5 = merged_df.groupby('number_challenge').apply(select_desired_row).reset_index(drop=True)
hawkeye_ids = david['HAWKEYE_ID'].unique()
number_hawkeye = relax5['number_hawkeye'].unique()

# Calculate the total number of unique values
total_unique_values = len(hawkeye_ids)

# Get the intersection of the two sets of unique values
common_values = len(set(hawkeye_ids) & set(number_hawkeye))

# Compute the accuracy
accuracy = common_values / total_unique_values

print(f"Accuracy: {accuracy * 100}%")
print(common_values)


Accuracy: 55.88235294117647%
19


In [1510]:
strict = strict[abs(strict['distance'] - strict['dist_abs']) <= 0.01]
relax1 = relax1[abs(relax1['distance'] - relax1['dist_abs']) <= 0.01]
relax2 = relax2[abs(relax2['distance'] - relax2['dist_abs']) <= 0.01]
relax3 = relax3[abs(relax3['distance'] - relax3['dist_abs']) <= 0.01]
relax4 = relax4[abs(relax4['distance'] - relax4['dist_abs']) <= 0.01]
relax5 = relax5[abs(relax5['distance'] - relax5['dist_abs']) <= 0.01]
strict_03 = strict[abs(strict['distance'] - strict['dist_abs']) <= 0.003]
relax1_03 = relax1[abs(relax1['distance'] - relax1['dist_abs']) <= 0.003]
relax2_03 = relax2[abs(relax2['distance'] - relax2['dist_abs']) <= 0.003]
strict['strict'] = 1
relax1['relax1'] = 1
relax2['relax2'] = 1
relax3['relax3'] = 1
relax4['relax4'] = 1
relax5['relax5'] = 1
strict_03['strict_03'] = 1
relax1_03['relax1_03'] = 1
relax2_03['relax2_03'] = 1

print(len(strict))
print(len(relax1))
print(len(relax2))
print(len(relax3))
print(len(relax4))
print(len(relax5))

print(len(strict_03))
print(len(relax1_03))
print(len(relax2_03))









20
27
30
31
33
34
9
15
16


/var/folders/2z/43q7xvvn577_kj388hr4w8zh0000gn/T/ipykernel_18433/678681737.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  strict['strict'] = 1
/var/folders/2z/43q7xvvn577_kj388hr4w8zh0000gn/T/ipykernel_18433/678681737.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  strict_03['strict_03'] = 1
/var/folders/2z/43q7xvvn577_kj388hr4w8zh0000gn/T/ipykernel_18433/678681737.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [1511]:
#strict_03 = strict
#relax1_03 = relax1
#relax2_03 = relax2


In [1512]:
#def process_df(df, value):
   # mask = abs(df['distance'] - df['dist_abs']) > value
    #df.loc[mask] = 0
    #return df

#strict = process_df(strict, 0.01)
#relax1 = process_df(relax1, 0.01)
#relax2 = process_df(relax2, 0.01)
#relax3 = process_df(relax3, 0.01)
#relax4 = process_df(relax4, 0.01)
#relax5 = process_df(relax5, 0.01)




In [1513]:
#def process_df(df, value):
    #mask = abs(df['distance'] - df['dist_abs']) > value
    #df.loc[mask] = 0
    #return df
#strict_03 = process_df(strict_03, 0.003)
#relax1_03 = process_df(relax1_03, 0.003)
#relax2_03 = process_df(relax2_03, 0.003)

In [1514]:
relax1

,server,receiver,scorer,impact,x1,y1,foldername_x,tournament_x,iiid,gender,scoreserver,scorereceiver,iid1,idmatch,idinpoint,bounce,iid2,scorer2,volley,puttheball,doublebounce,alreadydoublebounce,issueplayerside,dist,out_x,alreadyout,errorscoregame,inside,player1,scoregameplayer1,scoregameplayer2,scoresetserver,scoresetreceiver,scoregameserver,scoregamereceiver,hits,set_x,sum_games_x,server_score,receiver_score,tournament_id,challenger1_id,round,opponent1_id,match_time,set1_chal,set1_opp,set2_chal,set2_opp,set3_chal,set3_opp,match_challenge_count,challenges_left,call_in,distance_mm,line,player_stopped,ump_comments,umpire_challenger_ord,tournament_y,challenger_name,opponent_name,min,max,foldername_y,serve,chal_served_point,call,distance,hit,number_hawkeye,number_challenge,dist_abs,score,game_chal,game_opp,hawk_in,challenge_won,comb_game_x,comb_game_y,relax1
0,LOPEZ,DAVYDENKO,-1,0.0,-6.651841,-1.147098,SemiLopezDavydenko,DubaiMens08,570970.0,male,1,3,127699.0,573,1,1,92500.0,-1,0,server,0,0,NaN,-0.123161,1,0,19.047619,0.0,0.0,3.0,1.0,1.0,1.0,1.0,2.0,LOPEZ,3.0,3.0,15,40,6,68.0,4,202.0,107,6.0,4.0,4.0,6.0,1.0,2.0,7,1,0,121.0,RS,0,,2,DubaiMens08,Feliciano LOPEZ,Nikolay Davydenko,68.0,202.0,SemiLopezDavydenko,1.0,1.0,out,0.121,LOPEZ,781,0,0.123161,1 3,15.0,40.0,0,0,55.0,55.0,1
1,DAVYDENKO,LOPEZ,1,0.0,-6.414097,2.683151,SemiLopezDavydenko,DubaiMens08,570793.0,male,1,0,127654.0,573,1,1,92466.0,1,0,server,0,0,NaN,0.124471,0,0,0.000000,1.0,1.0,5.0,4.0,0.0,1.0,4.0,4.0,DAVYDENKO,2.0,8.0,15,0,6,68.0,4,202.0,82,6.0,4.0,4.0,4.0,NaN,NaN,4,2,1,132.0,RS,0,,2,DubaiMens08,Feliciano LOPEZ,Nikolay Davydenko,68.0,202.0,SemiLopezDavydenko,1.0,0.0,in,0.132,DAVYDENKO,604,1,0.124471,1 0,0.0,15.0,1,0,15.0,15.0,1
2,LOPEZ,DAVYDENKO,0,0.0,6.629273,3.591878,SemiLopezDavydenko,DubaiMens08,570225.0,male,0,2,127501.0,573,1,1,92358.0,0,0,server,0,0,NaN,-0.092159,1,0,18.181818,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,LOPEZ,1.0,1.0,0,30,6,68.0,4,202.0,4,0.0,1.0,NaN,NaN,NaN,NaN,1,2,0,86.0,LS,0,,2,DubaiMens08,Feliciano LOPEZ,Nikolay Davydenko,68.0,202.0,SemiLopezDavydenko,1.0,1.0,out,0.086,LOPEZ,37,2,0.092159,0 2,0.0,30.0,0,0,30.0,30.0,1
4,DAVYDENKO,LOPEZ,-1,18.0,10.686390,-4.157564,SemiLopezDavydenko,DubaiMens08,570420.0,male,0,1,127553.0,573,7,7,92395.0,-1,0,server,0,0,NaN,-0.009434,1,0,0.000000,0.0,1.0,4.0,5.0,0.0,0.0,4.0,4.0,DAVYDENKO,1.0,8.0,0,15,6,202.0,4,68.0,30,4.0,4.0,NaN,NaN,NaN,NaN,2,1,0,6.0,LF,0,,1,DubaiMens08,Nikolay Davydenko,Feliciano LOPEZ,68.0,202.0,SemiLopezDavydenko,0.0,NaN,out,0.006,DAVYDENKO,231,4,0.009434,0 1,0.0,15.0,0,0,15.0,15.0,1
5,LOPEZ,DAVYDENKO,-1,12.0,-9.783928,4.180157,SemiLopezDavydenko,DubaiMens08,570958.0,male,0,0,127694.0,573,5,5,92495.0,-1,0,server,0,0,NaN,-0.029688,1,0,19.047619,0.0,0.0,3.0,1.0,1.0,1.0,1.0,2.0,LOPEZ,3.0,3.0,0,0,6,68.0,4,202.0,105,6.0,4.0,4.0,6.0,1.0,2.0,6,2,0,26.0,RN,1,,2,DubaiMens08,Feliciano LOPEZ,Nikolay Davydenko,68.0,202.0,SemiLopezDavydenko,0.0,NaN,out,0.026,LOPEZ,769,5,0.029688,0 0,NaN,NaN,0,0,0.0,0.0,1
6,DAVYDENKO,LOPEZ,0,0.0,-6.606727,3.722899,SemiLopezDavydenko,DubaiMens08,570531.0,male,2,3,127582.0,573,1,1,92415.0,1,0,server,0,0,NaN,-0.074273,1,0,0.000000,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,DAVYDENKO,2.0,0.0,30,40,6,68.0,4,202.0,46,NaN,NaN,6.0,4.0,NaN,NaN,3,2,1,66.0,RS,0,,2,DubaiMens08,Feliciano LOPEZ,Nikolay Davydenko,68.0,202.0,SemiLopezDavydenko,1.0,0.0,out,0.066,DAVYDENKO,342,6,0.074273,2 3,40.0,30.0,0,1,70.0,70.0,1
9,DAVYDENKO,LOPEZ,-1,51.0,5.884794,4.137183,SemiLopezDavydenko,DubaiMens08,570510.0,male,0,1,127577.0,573,18,18,92411.0,-1,0,receiver,0,0,NaN,-0.000354,1,0,0.000000,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,LOPEZ,2.0,0.0,0,15,6,202.0,4,68.0,44,NaN,NaN,4.0,6.0,NaN,NaN,3,2,1,5.0,LN,0,,1,DubaiMens08,Nikolay Davydenko,Feliciano LOPEZ,68.0,202.0,SemiLopezDavydenko,0.0,NaN,in,0.005,LOPEZ,321,9,0.000354,0 1,15.0,0.0,1,0,15.0,15.0,1
10,LOPEZ,DAVYDENKO,1,0.0,-5.593488,-0.014768,SemiLopezDavydenko,DubaiMens08,570255.0,male,1,2,127514.0,573,1,1,92367.0,1,0,server,0,0,NaN,0.034284,0,0,0.000000,1.0,0.0,3.0,1.0,0.0,0.0,0.0,

In [1515]:
dataframes = [strict, relax1, relax2, relax3, relax4, relax5]
names = ['strict', 'relax1', 'relax2', 'relax3', 'relax4', 'relax5']

merged = pd.DataFrame()

for df, name in zip(dataframes, names):
    df = df[['number_hawkeye']].rename(columns={'number_hawkeye': name})
    if merged.empty:
        merged = df
    else:
        merged = pd.merge(merged, df, left_index=True, right_index=True)

In [1516]:
merged

,strict,relax1,relax2,relax3,relax4,relax5
0,781,781,781,781,924,924
1,604,604,604,604,604,604
2,37,37,37,37,37,37
4,231,231,231,231,231,231
5,769,769,769,769,769,769
6,342,342,342,320,423,843
9,321,321,321,321,321,812
10,67,67,67,67,67,505
12,1432,1432,1432,1432,1432,1432
16,1428,1428,1428,1428,1428,2084


In [1517]:
merged.to_csv('/Users/lucaslippman/Daniel/new_rows1.csv')

In [1518]:
strict_03 = strict
relax1_03 = relax1
relax2_03 = relax2


def process_df(df, value):
    mask = abs(df['distance'] - df['dist_abs']) > value
    df.loc[mask] = 0
    return df
strict_03 = process_df(strict_03, 0.003)
relax1_03 = process_df(relax1_03, 0.003)
relax2_03 = process_df(relax2_03, 0.003)

In [1519]:
dataframes = [strict_03, relax1_03, relax2_03]
names = ['strict_03', 'relax1_03', 'relax2_03']

merged = pd.DataFrame()

for df, name in zip(dataframes, names):
    df = df[['number_hawkeye']].rename(columns={'number_hawkeye': name})
    if merged.empty:
        merged = df
    else:
        merged = pd.merge(merged, df, left_index=True, right_index=True)

In [1520]:
merged

,strict_03,relax1_03,relax2_03
0,781,781,781
1,0,0,0
2,0,0,0
4,0,0,0
5,0,0,0
6,0,0,0
9,0,0,0
10,67,67,67
12,1432,1432,1432
16,1428,1428,1428


In [1521]:
merged.to_csv('/Users/lucaslippman/Daniel/new_rows2.csv')

In [1522]:
joint_merge1 = strict[['number_hawkeye']]
joint_merge2 = relax1[['number_hawkeye']]
joint_merge3 = relax2[['number_hawkeye']]
joint_merge4 = relax3[['number_hawkeye']]
joint_merge5 = relax4[['number_hawkeye']]
joint_merge6 = relax5[['number_hawkeye']]

In [1523]:
strict = strict[['strict', 'number_hawkeye']]
relax1 = relax1[['relax1', 'number_hawkeye']]
relax2 = relax2[['relax2', 'number_hawkeye']]
relax3 = relax3[['relax3', 'number_hawkeye']]
relax4 = relax4[['relax4', 'number_hawkeye']]
relax5 = relax5[['relax5', 'number_hawkeye']]
strict_03 = strict_03[['strict_03', 'number_hawkeye']]
relax1_03 = relax1_03[['relax1_03', 'number_hawkeye']]
relax2_03 = relax2_03[['relax2_03', 'number_hawkeye']]


KeyError: "['strict_03'] not in index"

In [ ]:
relax2_03

,relax2_03,number_hawkeye
0,1,781
10,1,67
11,1,1905
12,1,1432
13,1,1482
16,1,1428
17,1,1767
18,1,1437
20,1,1892
21,1,1298


In [ ]:
merged = strict.merge(hawkeye, how='right', left_on='number_hawkeye', right_index=True)

# Identify common columns
common_cols = strict.columns.intersection(hawkeye.columns)

# Reset the index
merged.reset_index(inplace=True, drop=True)

merged = merged.drop('number_hawkeye_x', axis=1)
merged['strict'] = merged['strict'].fillna(0)




In [ ]:
merged = relax1.merge(merged, how='right', left_on='number_hawkeye', right_index=True)

# Identify common columns
common_cols = relax1.columns.intersection(hawkeye.columns)

# Reset the index
merged.reset_index(inplace=True, drop=True)
merged = merged.drop('number_hawkeye_x', axis=1)
merged['relax1'] = merged['relax1'].fillna(0)



/var/folders/2z/43q7xvvn577_kj388hr4w8zh0000gn/T/ipykernel_18433/2360971387.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'number_hawkeye_y'} in the result is deprecated and will raise a MergeError in a future version.
  merged = relax1.merge(merged, how='right', left_on='number_hawkeye', right_index=True)


In [ ]:
merged = relax2.merge(merged, how='right', left_on='number_hawkeye', right_index=True)

# Identify common columns
common_cols = relax2.columns.intersection(hawkeye.columns)

# Reset the index
merged.reset_index(inplace=True, drop=True)
merged = merged.drop('number_hawkeye_x', axis=1)
merged['relax2'] = merged['relax2'].fillna(0)


/var/folders/2z/43q7xvvn577_kj388hr4w8zh0000gn/T/ipykernel_18433/2246911802.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'number_hawkeye_y'} in the result is deprecated and will raise a MergeError in a future version.
  merged = relax2.merge(merged, how='right', left_on='number_hawkeye', right_index=True)


In [ ]:
merged = relax3.merge(merged, how='right', left_on='number_hawkeye', right_index=True)

# Identify common columns
common_cols = relax3.columns.intersection(hawkeye.columns)

# Reset the index
merged.reset_index(inplace=True, drop=True)
merged = merged.drop('number_hawkeye_x', axis=1)
merged['relax3'] = merged['relax3'].fillna(0)


/var/folders/2z/43q7xvvn577_kj388hr4w8zh0000gn/T/ipykernel_18433/1503434192.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'number_hawkeye_y'} in the result is deprecated and will raise a MergeError in a future version.
  merged = relax3.merge(merged, how='right', left_on='number_hawkeye', right_index=True)


In [ ]:
merged = relax4.merge(merged, how='right', left_on='number_hawkeye', right_index=True)

# Identify common columns
common_cols = relax4.columns.intersection(hawkeye.columns)

# Reset the index
merged.reset_index(inplace=True, drop=True)
merged = merged.drop('number_hawkeye_x', axis=1)
merged['relax4'] = merged['relax4'].fillna(0)


/var/folders/2z/43q7xvvn577_kj388hr4w8zh0000gn/T/ipykernel_18433/721190914.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'number_hawkeye_y'} in the result is deprecated and will raise a MergeError in a future version.
  merged = relax4.merge(merged, how='right', left_on='number_hawkeye', right_index=True)


In [ ]:
merged = relax5.merge(merged, how='right', left_on='number_hawkeye', right_index=True)

# Identify common columns
common_cols = relax5.columns.intersection(hawkeye.columns)

# Reset the index
merged.reset_index(inplace=True, drop=True)
merged = merged.drop('number_hawkeye_x', axis=1)
merged['relax5'] = merged['relax5'].fillna(0)


/var/folders/2z/43q7xvvn577_kj388hr4w8zh0000gn/T/ipykernel_18433/233152353.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'number_hawkeye_y'} in the result is deprecated and will raise a MergeError in a future version.
  merged = relax5.merge(merged, how='right', left_on='number_hawkeye', right_index=True)


In [ ]:
merged = relax1_03.merge(merged, how='right', left_on='number_hawkeye', right_index=True)

# Identify common columns
common_cols = relax5.columns.intersection(hawkeye.columns)

# Reset the index
merged.reset_index(inplace=True, drop=True)
merged = merged.drop('number_hawkeye_x', axis=1)
merged['relax1_03'] = merged['relax1_03'].fillna(0)


/var/folders/2z/43q7xvvn577_kj388hr4w8zh0000gn/T/ipykernel_18433/2104706827.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'number_hawkeye_y'} in the result is deprecated and will raise a MergeError in a future version.
  merged = relax1_03.merge(merged, how='right', left_on='number_hawkeye', right_index=True)


In [ ]:
merged = strict_03.merge(merged, how='right', left_on='number_hawkeye', right_index=True)

# Identify common columns
common_cols = relax5.columns.intersection(hawkeye.columns)

# Reset the index
merged.reset_index(inplace=True, drop=True)
merged = merged.drop('number_hawkeye_x', axis=1)
merged['strict_03'] = merged['strict_03'].fillna(0)


/var/folders/2z/43q7xvvn577_kj388hr4w8zh0000gn/T/ipykernel_18433/2686672240.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'number_hawkeye_y'} in the result is deprecated and will raise a MergeError in a future version.
  merged = strict_03.merge(merged, how='right', left_on='number_hawkeye', right_index=True)


In [ ]:
merged = relax2_03.merge(merged, how='right', left_on='number_hawkeye', right_index=True)

# Identify common columns
common_cols = relax5.columns.intersection(hawkeye.columns)

# Reset the index
merged.reset_index(inplace=True, drop=True)
merged = merged.drop('number_hawkeye_x', axis=1)
merged['relax2_03'] = merged['relax2_03'].fillna(0)


/var/folders/2z/43q7xvvn577_kj388hr4w8zh0000gn/T/ipykernel_18433/3766047147.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'number_hawkeye_y'} in the result is deprecated and will raise a MergeError in a future version.
  merged = relax2_03.merge(merged, how='right', left_on='number_hawkeye', right_index=True)


In [ ]:
merged = merged.drop('number_hawkeye_y', axis=1)
merged = merged.drop('number_hawkeye', axis=1)
merged



,relax2_03,strict_03,relax1_03,relax5,relax4,relax3,relax2,relax1,strict,server,receiver,score,scorer,impact,x1,y1,foldername,tournament,iiid,gender,scoreserver,scorereceiver,iid1,idmatch,idinpoint,bounce,iid2,scorer2,volley,puttheball,doublebounce,alreadydoublebounce,issueplayerside,dist,out,alreadyout,errorscoregame,inside,player1,scoregameplayer1,scoregameplayer2,scoresetserver,scoresetreceiver,scoregameserver,scoregamereceiver,hits,set,sum_games,dist_abs,server_score,receiver_score,comb_game_x,key
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DAVYDENKO,LOPEZ,0 0,1,0.0,4.677922,2.536106,SemiLopezDavydenko,DubaiMens08,570188.0,male,0,0,127491.0,573,1,1,92350.0,1,0,server,0,0,NaN,1.623120,0,0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,DAVYDENKO,1.0,0.0,1.623120,0,0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DAVYDENKO,LOPEZ,0 0,1,3.0,-6.956489,-0.608451,SemiLopezDavydenko,DubaiMens08,570189.0,male,0,0,127491.0,573,2,2,92350.0,1,0,receiver,0,0,NaN,3.540612,0,0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,LOPEZ,1.0,0.0,3.540612,0,0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DAVYDENKO,LOPEZ,0 0,1,6.0,10.268340,-3.711434,SemiLopezDavydenko,DubaiMens08,570190.0,male,0,0,127491.0,573,3,3,92350.0,1,0,server,0,0,NaN,0.432145,0,0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,DAVYDENKO,1.0,0.0,0.432145,0,0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DAVYDENKO,LOPEZ,1 0,-1,0.0,4.317173,-1.403491,SemiLopezDavydenko,DubaiMens08,570191.0,male,1,0,127492.0,573,1,1,92351.0,-1,0,server,0,0,NaN,1.447192,0,0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,DAVYDENKO,1.0,0.0,1.447192,15,0,15.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DAVYDENKO,LOPEZ,1 0,-1,3.0,-8.256093,-0.687064,SemiLopezDavydenko,DubaiMens08,570192.0,male,1,0,127492.0,573,2,2,92351.0,-1,0,receiver,0,0,NaN,3.446542,0,0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,LOPEZ,1.0,0.0,3.446542,15,0,15.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,MURRAY,ANCIC,1 0,1,18.0,-8.584010,-3.564925,FinalMurayAncic,Marseille08,1483786.0,male,1,0,335677.0,1543,7,7,243009.0,1,0,server,0,0,NaN,0.570891,0,0,0.0,1.0,0.0,4.0,6.0,1.0,0.0,5.0,4.0,MURRAY,2.0,9.0,0.570891,15,0,15.0,1
2989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,MURRAY,ANCIC,2 0,1,0.0,-4.783113,-3.078492,FinalMurayAncic,Marseille08,1483787.0,male,2,0,335678.0,1543,1,1,243010.0,1,0,server,0,0,NaN,1.087810,0,0,0.0,1.0,0.0,4.0,6.0,1.0,0.0,5.0,4.0,MURRAY,2.0,9.0,1.087810,30,0,30.0,1
2990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,MURRAY,ANCIC,2 0,1,3.0,12.249090,0.208100,FinalMurayAncic,Marseille08,1483788.0,male,2,0,335678.0,1543,2,2,243010.0,1,0,receiver,0,0,NaN,-0.271700,1,0,0.0,0.0,0.0,4.0,6.0,1.0,0.0,5.0,4.0,ANCIC,2.0,9.0,0.271700,30,0,30.0,1
2991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,MURRAY,ANCIC,3 0,1,0.0,-6.349992,0.644934,FinalMurayAncic,Marseille08,1483789.0,male,3,0,335679.0,1543,1,1,243011.0,1,0,server,0,0,NaN,0.190664,0,0,0.0,1.0,0.0,4.0,6.0,1.0,0.0,5.0,4.0,MURRAY,2.0,9.0,0.190664,40,0,40.0,1


In [ ]:
cols_to_move = ['strict', 'relax1', 'relax2','relax3','relax4','relax5','strict_03','relax1_03','relax2_03']

cols_remain = [col for col in merged.columns if col not in cols_to_move]

# rearrange the columns
merged = merged[cols_remain + cols_to_move]


In [ ]:
merged.to_csv('/Users/lucaslippman/Daniel/rows_updated.csv')
